## Ucitavanje i Sredjivanje Nizova Podataka

In [1]:
#ucitavanje potrebnih biblioteka
import pandas as pd
import numpy as np

In [12]:
#uvoz podataka
#sezona 23/24
df_23_4 = pd.read_csv("serie-a-2023.csv")
#sezona 24/25
df_24_5 = pd.read_csv("serie-a-2024.csv")
#sezona 25/6
df_25_6 = pd.read_csv("serie-a-2025.csv")

## Određivanje Duzine Trenutnoj Sezoni i Prikaz

In [13]:
#odrediti duzinu sezone - OBAVEZNO!!!
df_25_6 = df_25_6.head(50)
df_25_6.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
45,49,5,28/09/2025 13:00,Olimpico,Roma,Hellas Verona,2 - 0
46,45,5,28/09/2025 16:00,Ettore Giardiniero - Via del mare,Lecce,Bologna,2 - 2
47,46,5,28/09/2025 18:45,San Siro,Milan,Napoli,2 - 1
48,47,5,29/09/2025 16:30,Ennio Tardini,Parma,Torino,2 - 1
49,43,5,29/09/2025 18:45,Luigi Ferraris,Genoa,Lazio,0 - 3


In [14]:
#sjedinjavanje ta tri df niza podataka - po osi jedan - odnosno uzduž
df_BL = pd.concat([df_23_4, df_24_5, df_25_6], axis=0)

In [15]:
#prikaz sjedinjenog niza podatak 
#tj. sve tri sezone u jednom nizu podataka u obliku df 
df_BL.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
45,49,5,28/09/2025 13:00,Olimpico,Roma,Hellas Verona,2 - 0
46,45,5,28/09/2025 16:00,Ettore Giardiniero - Via del mare,Lecce,Bologna,2 - 2
47,46,5,28/09/2025 18:45,San Siro,Milan,Napoli,2 - 1
48,47,5,29/09/2025 16:30,Ennio Tardini,Parma,Torino,2 - 1
49,43,5,29/09/2025 18:45,Luigi Ferraris,Genoa,Lazio,0 - 3


## Predprocesovanje Niza Podataka

In [16]:
#Izbacivanje Nerelevantnih Osobina
df_BL = df_BL.drop(["Match Number","Round Number","Date","Location"],axis=1)
df_BL.shape

(810, 3)

In [17]:
df_BL.rename(columns={"Home Team":"H","Away Team":"A"},inplace=True)

In [18]:
df_BL

,H,A,Result
0,Empoli,Hellas Verona,0 - 1
1,Frosinone,Napoli,1 - 3
2,Genoa,Fiorentina,1 - 4
3,Inter,Monza,2 - 0
4,Roma,Salernitana,2 - 2
...,...,...,...
45,Roma,Hellas Verona,2 - 0
46,Lecce,Bologna,2 - 2
47,Milan,Napoli,2 - 1
48,Parma,Torino,2 - 1


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

#One Hot Encoder Timova - Domacin i Gost
onehot_encoder_domacin = OneHotEncoder(handle_unknown="ignore",drop="first")  # Ensure parameters are correct
kodiran_domacin = onehot_encoder_domacin.fit_transform(df_BL[["H"]]).toarray()
df_domacin = pd.DataFrame(kodiran_domacin,columns=onehot_encoder_domacin.get_feature_names_out(["H"]))

In [20]:
df_domacin.columns

Index(['H_Bologna', 'H_Cagliari', 'H_Como', 'H_Cremonese', 'H_Empoli',
       'H_Fiorentina', 'H_Frosinone', 'H_Genoa', 'H_Hellas Verona', 'H_Inter',
       'H_Juventus', 'H_Lazio', 'H_Lecce', 'H_Milan', 'H_Monza', 'H_Napoli',
       'H_Parma', 'H_Pisa', 'H_Roma', 'H_Salernitana', 'H_Sassuolo',
       'H_Torino', 'H_Udinese', 'H_Venezia'],
      dtype='object')

In [21]:
df_domacin.shape

(810, 24)

In [22]:
#One Hot Encoder Timova - Gosti
onehot_encoder_gost = OneHotEncoder(handle_unknown="ignore",drop="first")  # Ensure parameters are correct
kodiran_gost = onehot_encoder_gost.fit_transform(df_BL[["A"]]).toarray()
df_gost = pd.DataFrame(kodiran_gost,columns=onehot_encoder_gost.get_feature_names_out(["A"]))

In [23]:
df_gost.shape

(810, 24)

In [24]:
#Sjedinjavanje tih osobina domacin i gost
df_jedan = pd.concat([df_domacin,df_gost],axis=1)
df_jedan

,H_Bologna,H_Cagliari,H_Como,H_Cremonese,H_Empoli,H_Fiorentina,H_Frosinone,H_Genoa,H_Hellas Verona,H_Inter,...,A_Monza,A_Napoli,A_Parma,A_Pisa,A_Roma,A_Salernitana,A_Sassuolo,A_Torino,A_Udinese,A_Venezia
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Transformisanje Rezultata u Varijablu - Y Varijable

#### Stvaranje Vrijednosti za Dvije Kolone iz Jedne Osobine/Kolone Rezultat

In [25]:
rez_dom = df_BL["Result"].str.slice(0,1)

In [26]:
rez_dom.head()

0    0
1    1
2    1
3    2
4    2
Name: Result, dtype: object

In [27]:
rez_gost = df_BL["Result"].str.slice(4,5)

In [28]:
rez_gost.head()

0    1
1    3
2    4
3    0
4    2
Name: Result, dtype: object

In [29]:
df_rezultatski = pd.concat([df_BL,rez_dom,rez_gost],axis=1)

In [30]:
#promjena naziva kolonama - spajanjem smo dobili kolone istih naziva
df_rezultatski.columns = ["H","A","Result","Rez1","Rez2"]
df_rezultatski

,H,A,Result,Rez1,Rez2
0,Empoli,Hellas Verona,0 - 1,0,1
1,Frosinone,Napoli,1 - 3,1,3
2,Genoa,Fiorentina,1 - 4,1,4
3,Inter,Monza,2 - 0,2,0
4,Roma,Salernitana,2 - 2,2,2
...,...,...,...,...,...
45,Roma,Hellas Verona,2 - 0,2,0
46,Lecce,Bologna,2 - 2,2,2
47,Milan,Napoli,2 - 1,2,1
48,Parma,Torino,2 - 1,2,1


In [31]:
#spremanje df_epl varijable za obradu osobine Result u excelu
df_rezultatski.to_csv("uredjivanje_skora_vani.csv")

## Uocavanje Indeksa Sezona - za Kasnije Floatiranje

In [32]:
#df_rezultatski[303:306] #sezona 23/4 od 0:306
#df_rezultatski[306:612] #sezona 24/5 od 306:612
#df_rezultatski[612:] #sezona25/6 od 612:

## Stvaranje Kolone Skor sa Vrijednostima - Ovdje

In [33]:
# Define a function to perform the conditional logic
def calculate_value(row):
    if row['Rez1'] > row['Rez2']:
        return 1
    elif row['Rez1'] < row['Rez2']:
        return -1
    else:  # row['Column1'] == row['Column2']
        return 0

In [34]:
# Apply the function to each row in the DataFrame and assign the result to a new column
df_rezultatski['Skor'] = df_rezultatski.apply(calculate_value, axis=1)
skor = df_rezultatski.drop(['H', 'A', 'Result', 'Rez1',"Rez2"],axis=1)

### Floatiranje Vrijednosti Kolone Skor - po Sezonama

In [35]:
#sezona 23/4
skor23_4 = skor[0:306]
skor24_5 = skor[306:612]
skor25_6 = skor[612:]

In [36]:
#skor.head(3)
skor23_4["Skor"] = skor23_4["Skor"] * 0.71

C:\Users\amirb\AppData\Local\Temp\ipykernel_6792\3544292283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skor23_4["Skor"] = skor23_4["Skor"] * 0.71


In [37]:
skor24_5["Skor"] = skor24_5["Skor"] * 0.85

C:\Users\amirb\AppData\Local\Temp\ipykernel_6792\3136881136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skor24_5["Skor"] = skor24_5["Skor"] * 0.85


## Konačna kolona Skor - Sjedinjavanje Vrijednosti Sezona

In [38]:
skor = pd.concat([skor23_4, skor24_5, skor25_6], axis=0, ignore_index=True)

In [39]:
df_rezultatski = df_rezultatski.reset_index(drop=True)
skor = skor.reset_index(drop=True)

In [40]:
#provjera rezultata
finalna_provjera = pd.concat([df_rezultatski,skor],axis=1,ignore_index=True)
finalna_provjera = finalna_provjera.drop([3,4,5],axis=1)
finalna_provjera.columns = ["H","A","Rezultat","Skor"]

In [41]:
finalna_provjera.head()

,H,A,Rezultat,Skor
0,Empoli,Hellas Verona,0 - 1,-0.71
1,Frosinone,Napoli,1 - 3,-0.71
2,Genoa,Fiorentina,1 - 4,-0.71
3,Inter,Monza,2 - 0,0.71
4,Roma,Salernitana,2 - 2,0.00


# Konačni df - Podaci su Sređeni

In [42]:
skor = skor.reset_index(drop=True)
df_jedan = df_jedan.reset_index(drop=True)

In [43]:
df_BL = pd.concat([df_jedan,skor],axis=1)

In [44]:
df_BL.head(5)

,H_Bologna,H_Cagliari,H_Como,H_Cremonese,H_Empoli,H_Fiorentina,H_Frosinone,H_Genoa,H_Hellas Verona,H_Inter,...,A_Napoli,A_Parma,A_Pisa,A_Roma,A_Salernitana,A_Sassuolo,A_Torino,A_Udinese,A_Venezia,Skor
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.71
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00


In [45]:
#spremanje kodiranog niza podataka
df_BL.to_csv("kodirani_bl.csv")

## Razdvajanje Niza Podataka na Osobine za Predvidjanje i Ciljanu Osobinu

In [46]:
# X predvidjacke, y varijabla jeste Skor Osobina
X = df_BL.drop(["Skor"],axis=1)
y = df_BL["Skor"]

In [47]:
#razdvajanje podataka u treniranju i testirajuce nizove
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=42)
X_train.head()

,H_Bologna,H_Cagliari,H_Como,H_Cremonese,H_Empoli,H_Fiorentina,H_Frosinone,H_Genoa,H_Hellas Verona,H_Inter,...,A_Monza,A_Napoli,A_Parma,A_Pisa,A_Roma,A_Salernitana,A_Sassuolo,A_Torino,A_Udinese,A_Venezia
432,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
493,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
#skaliranje ovih osobina
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[-0.23123194, -0.22457132, -0.16795459, ..., -0.23123194,
        -0.23450536, -0.15911146],
       [-0.23123194, -0.22457132, -0.16795459, ..., -0.23123194,
        -0.23450536, -0.15911146],
       [-0.23123194, -0.22457132, -0.16795459, ..., -0.23123194,
        -0.23450536, -0.15911146],
       ...,
       [-0.23123194, -0.22457132, -0.16795459, ...,  4.32466215,
        -0.23450536, -0.15911146],
       [-0.23123194, -0.22457132, -0.16795459, ..., -0.23123194,
        -0.23450536,  6.28490254],
       [-0.23123194, -0.22457132, -0.16795459, ..., -0.23123194,
        -0.23450536, -0.15911146]])

In [49]:
#Spremanje tog Kodera - za Kasniju Upotrebu

with open("onehot_encoder_domacin.pkl","wb") as file:
    pickle.dump(onehot_encoder_domacin,file)
    
with open("onehot_encoder_gost.pkl","wb") as file:
    pickle.dump(onehot_encoder_gost,file)
    
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)
    

## Stvaranje Modela i Dobijanje Predikcije

In [50]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
#sledeca biblioteka se odnosi na stvaranje neurona iz kutije layers i alatom dense
from tensorflow.keras.layers import Dense

In [51]:
#konstrukcija modela
model=Sequential([
    Dense(16,activation="relu",input_shape=(X_train.shape[1],)), #HL1 connected to I/p - Skriveni Sloj 1 povezan sa I/p
    Dense(1,activation="tanh")
])

In [52]:
#Oblikovanje Modela 
model.compile(optimizer="adam",loss="mean_absolute_error",metrics=["mae"])
model.summary()

# Keras dokumentacija pojasnjava sta se dogodi sa razlicitim odabirom "loss" i drugih parametara

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                784       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [53]:
import datetime
#uspostavljanje Tensorboard-a, zapocinjanje treniranja
#TensorBoard sluzi samo za vizualizaciju svih logovanja koje izvrsim kod treniranja nekog modela
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

#log_dir varijabla koja osigurava da ostane zabinjezen trag svakog treniranja modela
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

#Uspostavljanje Early Stopping - sluzi za zaustavljanje treniranja ako se vrijednost gubitaka ne smanjuje
#bitni parametri su uneseni koji sluze poboljsanju funkcije ranog zaustavljanja treniranja modela...
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)


In [54]:
#TRENIRANJE MODELA / Tovarenje Modela
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=50,
    callbacks= [tensorflow_callback,early_stopping_callback]
)

Epoch 1/50
23/23 [==============================] - 4s 55ms/step - loss: 0.7164 - mae: 0.7164 - val_loss: 0.7111 - val_mae: 0.7111
Epoch 2/50
23/23 [==============================] - 1s 25ms/step - loss: 0.6733 - mae: 0.6733 - val_loss: 0.6857 - val_mae: 0.6857
Epoch 3/50
23/23 [==============================] - 1s 33ms/step - loss: 0.6399 - mae: 0.6399 - val_loss: 0.6563 - val_mae: 0.6563
Epoch 4/50
23/23 [==============================] - 1s 24ms/step - loss: 0.6124 - mae: 0.6124 - val_loss: 0.6278 - val_mae: 0.6278
Epoch 5/50
23/23 [==============================] - 1s 60ms/step - loss: 0.5857 - mae: 0.5857 - val_loss: 0.6052 - val_mae: 0.6052
Epoch 6/50
23/23 [==============================] - 1s 25ms/step - loss: 0.5635 - mae: 0.5635 - val_loss: 0.5788 - val_mae: 0.5788
Epoch 7/50
23/23 [==============================] - 0s 20ms/step - loss: 0.5447 - mae: 0.5447 - val_loss: 0.5747 - val_mae: 0.5747
Epoch 8/50
23/23 [==============================] - 1s 28ms/step - loss: 0.5315 - m

In [55]:
#Procjena Rada Modela preko test podataka
test_loss, test_mae = model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 6ms/step - loss: 0.5747 - mae: 0.5747


In [56]:
#spremanje modela
model.save("sa_float_model.h5")

In [57]:
#moze biti da skaliranje je neophodno, onda je potrebno i obnoviti test parametara modela, te iznova
#-trenirati model i na kraju ga ponovo spremiti.